<a href="https://colab.research.google.com/github/ekrombouts/GenCareAI/blob/main/scripts/data_generation/110_GenerateClientProfilesWithLangchain.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# GenCare AI: Generating client profiles

**Author:** Eva Rombouts  
**Date:** 2024-06-01  
**Version:** 1.0

### Description
This script generates synthetic healthcare data for NLP experiments.  
It utilizes OpenAI to create an anonymized dataset that mimics real-world client records of nursing care homes for use in machine learning and data analysis.

### Setup
- Set the [OpenAI API key](https://platform.openai.com/docs/quickstart?context=python) in the environment or configuration file.
- Mount Google Drive for file operations if running in Colab.


In [40]:
# Imports needed for Google Colab
!pip install -q langchain langchain_core langchain_openai

In [41]:
from typing import List

from langchain.output_parsers import PydanticOutputParser
from langchain_core.prompts import PromptTemplate
from langchain_core.pydantic_v1 import BaseModel, Field
from langchain_openai import ChatOpenAI

import pandas as pd
import os

In [42]:
# Mount google drive for storing the .csv file
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [43]:
# Get the OpenAI API key.
from google.colab import userdata
OPENAI_API_KEY =userdata.get('GCI_OPENAI_API_KEY')

When using the OpenAI model with the same inputs, it tends to produce similar outputs each time (a phenomenon known as determinism). This is a limitation in this scenario where we aim for diversity and variability in generated results. Even with the temperature set to 1 to increase variability, the model frequently selected Lewy Body dementia as the dementia type and diabetes, arthritis, and hypertension as conditions.

To enhance diversity, I implemented several strategies:
- Switching to the gpt-4o model
- Creating multiple profiles in each query
- Adjusting the temperature setting to 1 to promote more varied responses.

In [57]:
# The ward name will be used in the filename. Practical when performing multiple
# experiments
WARD_NAME = 'Horizon'
# Per query eight profiles are generated. The query is run NUM_WINGS times, so
# when NUM_WINGS is set to 3 the total number of client profiles generated is 24.
NUM_WINGS = 3
# GPT-4o yields better, more diverse results and handles instructions better.
MODEL_PROFILES = 'gpt-4o-2024-05-13'
TEMP = 1.0

DATA_DIR = '/content/drive/My Drive/Colab Notebooks/GenCareAI/data'
FN_PROFILES = os.path.join(DATA_DIR, f'gcai_client_profiles_{WARD_NAME}.csv')

In [58]:
# Initialize OpenAI model
model = ChatOpenAI(api_key=OPENAI_API_KEY, temperature=TEMP, model=MODEL_PROFILES)

In [59]:
# Define a Pydantic model to structure the client profile data
class ClientProfile(BaseModel):
    naam: str = Field(description="naam van de client (Meneer/Mevrouw Voornaam Achternaam, wees creatief in de namen)")
    type_dementie: str = Field(description="type dementie (Alzheimer, gemengde dementie, vasculaire dementie, lewy body dementie, parkinsondementie, FTD: varieer, de kans op Alzheimer, gemengde en vasculaire dementie is het grootst)")
    somatiek: str = Field(description="lichamelijke klachten")
    # biografie: str = Field(description="een korte beschrijving van karakter en relevante biografische gegevens (vermijd stereotypen in beroep en achtergrond)")
    adl: str = Field(description="beschrijf welke ADL hulp de cliënt nodig heeft")
    mobiliteit: str = Field(description="beschrijf de mobiliteit (bv rolstoelafhankelijk, gebruik rollator, valgevaar)")
    gedrag: str = Field(description="beschrijf voor de zorg relevante aspecten van cognitie en probleemgedrag. Varieer met de ernst van het probleemgedrag van rustige cliënten, gemiddeld onrustige cliënten tot cliënten die fors apathisch, onrustig, angstig, geagiteerd of zelfs agressief kunnen zijn")

# Define a Pydantic model to hold multiple client profiles
class ClientProfiles(BaseModel):
    clients: List[ClientProfile]

In [60]:
# Query to generate multiple client profiles
client_query = """
Schrijf acht profielen van cliënten die zijn opgenomen op een psychogeriatrische afdeling van het verpleeghuis. Hier wonen mensen met een gevorderde dementie met een hoge zorgzwaarte.
Zorg dat de profielen erg van elkaar verschillen.
"""

In [61]:
# Set up a parser to handle the output and inject instructions into the prompt template
parser = PydanticOutputParser(pydantic_object=ClientProfiles)

In [62]:
# Define the prompt template with placeholders for query and format instructions
prompt = PromptTemplate(
    template="Beantwoord de query.\n{format_instructions}\n{query}\n",
    input_variables=["query"],
    partial_variables={"format_instructions": parser.get_format_instructions()},
)

# Check the prompt template
print(prompt)

input_variables=['query'] partial_variables={'format_instructions': 'The output should be formatted as a JSON instance that conforms to the JSON schema below.\n\nAs an example, for the schema {"properties": {"foo": {"title": "Foo", "description": "a list of strings", "type": "array", "items": {"type": "string"}}}, "required": ["foo"]}\nthe object {"foo": ["bar", "baz"]} is a well-formatted instance of the schema. The object {"properties": {"foo": ["bar", "baz"]}} is not well-formatted.\n\nHere is the output schema:\n```\n{"properties": {"clients": {"title": "Clients", "type": "array", "items": {"$ref": "#/definitions/ClientProfile"}}}, "required": ["clients"], "definitions": {"ClientProfile": {"title": "ClientProfile", "type": "object", "properties": {"naam": {"title": "Naam", "description": "naam van de client (Meneer/Mevrouw Voornaam Achternaam, wees creatief in de namen)", "type": "string"}, "type_dementie": {"title": "Type Dementie", "description": "type dementie (Alzheimer, gemeng

In [63]:
# Create a chain of operations: prompt template -> model -> output parser
chain = prompt | model | parser

In [64]:
def load_or_generate_data(file_path):
    """Load data from file if exists, otherwise generate new data."""
    try:
        if os.path.exists(file_path):
            df = pd.read_csv(file_path)
            print("Data file found. Data loaded successfully.")
        else:
            print("Data file not found. Generating new data...")

            all_data = []
            for _ in range(NUM_WINGS):
                result = chain.invoke({"query": client_query})
                if result is None or not hasattr(result, 'clients'):
                    raise ValueError("No valid response received from the model.")
                data = [client.dict() for client in result.clients]
                all_data.extend(data)
            df = pd.DataFrame(all_data)
            df.to_csv(file_path, index=False)
            print("Data generated and saved successfully.")
        return df

    except Exception as e:
        print(f"An error occurred: {e}")
        raise


In [65]:
# Main execution
df = load_or_generate_data(FN_PROFILES)

Data file not found. Generating new data...
Data generated and saved successfully.


In [66]:
df.head(24)

,naam,type_dementie,somatiek,adl,mobiliteit,gedrag
0,Meneer Jan de Vries,Alzheimer,"Beperkt gehoor, artrose in beide knieën","Hulp bij aankleden en wassen, eet zelfstandig","Maakt gebruik van een rollator, valgevaarlijk","Redelijk rustig, soms verward en vergeetachtig"
1,Mevrouw Anna van den Berg,Lewy body dementie,"Beperkt zicht, diabetes type 2","Volledige hulp nodig bij eten en drinken, inco...","Rolstoelafhankelijk, kan niet zelfstandig lopen","Vaak angstig en hallucinaties, af en toe agres..."
2,Meneer Pieter Jansen,Vasculaire dementie,"Hartaandoening, hoge bloeddruk",Hulp nodig bij douchen en aan- en uitkleden,"Heeft een looprek nodig, langzaam en voorzicht...",Vertoont rustige momenten afgewisseld met peri...
3,Mevrouw Maria de Wit,Frontotemporale dementie (FTD),"Osteoporose, moeite met slikken",Hulp nodig bij alle dagelijkse handelingen,"Valgevaarlijk, soms bedlegerig","Sterk apathisch, soms plotseling boos of gefru..."
4,Meneer Kees Kok,Gemengde dementie,"COPD, slechthorend",Heeft hulp nodig bij het wassen en toiletbezoek,"Heeft een rollator nodig, kortademig bij inspa...","Vaak onrustig, begrijpt instructies slecht, ka..."
5,Mevrouw Lisa de Groot,Parkinsondementie,"Tremoren door Parkinson, moeite met fijne moto...","Hulp bij eten en drinken, hulp bij persoonlijk...","Rolstoelafhankelijk, trage mobiliteit","Rustig en volgzaam, maar snel verward en verge..."
6,Meneer Theo Peters,Alzheimer,"Chronische rugpijn, hypertensie","Heeft hulp nodig bij aan- en uitkleden, kan zi...","Valgevaarlijk, gebruikt een wandelstok","Apathisch, kan soms van streek raken, neigt af..."
7,Mevrouw Clara Smits,Vasculaire dementie,"Hartproblemen, nierfalen",Volledige hulp nodig bij voeding en toiletgang,"Bedlegerig, heeft fysiotherapie nodig","Vaak angstig en onrustig, moeite om gebeurteni..."
8,Meneer Jan de Vries,Alzheimer,Chronische rugpijn en artritis,Heeft hulp nodig bij het aankleden en in bad gaan,"Gebruik van een rollator, valgevaar","Rustig, maar kan verward zijn in de avonduren"
9,Mevrouw Anna Jansen,Vasculaire dementie,Diabetes en hoge bloeddruk,Heeft hulp nodig bij het klaarmaken van maalti...,Rolstoelafhankelijk,Apathisch en vertoont weinig initiatief
